In [163]:
import os
import numpy as np
import pandas as pd
from statsmodels.formula.api import OLS
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.discriminant_analysis import *
from sklearn.metrics import *
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Concatenate, Add, Reshape, Average
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

In [164]:
df=pd.read_excel("alldata.xls")
train=pd.read_excel("train.xls")
test=pd.read_excel("test.xls")

In [167]:
df=df.drop(["SignDate" , "as_PayOffDate" , "as_PrPayOffDate", "DOB","as_Sex","yhat","age","as_TotOvrdDays","default_cont"],axis=1)
train=train.drop(["SignDate" , "as_PayOffDate" , "as_PrPayOffDate", "DOB","as_Sex","yhat","age","as_TotOvrdDays","default_cont"],axis=1)
test=test.drop(["SignDate" , "as_PayOffDate" , "as_PrPayOffDate", "DOB","as_Sex","yhat","age","as_TotOvrdDays",
     "default_cont"],axis=1)

In [168]:
df=df.dropna()
train=train.dropna()
test=test.dropna()

In [169]:
print(len(df))
print(len(train))
print(len(test))

9213
6868
2345


In [170]:
Y = df.default
X = df.drop("default",axis=1)
ids = [i for i in X.columns.tolist() if i[-2:]=="ID" or "Ovrd" in i]
X = X.drop(ids,axis=1)

In [172]:
Y_train = train.default
X_train = train.drop("default",axis=1)
ids = [i for i in X_train.columns.tolist() if i[-2:]=="ID" or "Ovrd" in i]
X_train= X_train.drop(ids,axis=1)


In [173]:
Y_test =test.default
X_test = test.drop("default",axis=1)
ids = [i for i in X_test.columns.tolist() if i[-2:]=="ID" or "Ovrd" in i]
X_test = X_test.drop(ids,axis=1)

In [175]:
def get_corr_cols(X,thresh=0.85):
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [i for i in upper.columns if any(upper[i] > thresh)]
    cols_no_corr = [i for i in X.columns if i not in to_drop]
    return cols_no_corr, to_drop

cols_no_corr, to_drop  = get_corr_cols(X)

# Neural Network

In [176]:
#MinMax scale X_train
scaler = MinMaxScaler(feature_range=[0,1])
scaler.fit(X_train[cols_no_corr])
X_train_scaled = pd.DataFrame(scaler.transform(X_train[cols_no_corr]), columns=X_train[cols_no_corr].columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test[cols_no_corr]), columns=X_test[cols_no_corr].columns)

In [177]:
y_train = to_categorical(Y_train, num_classes=2)
y_test = to_categorical(Y_test, num_classes=2)

In [178]:
%%time
input_shape = X_train_scaled.shape[1]

model = Sequential()
model.add(Dense(32, activation='sigmoid', input_shape=((input_shape,))))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
model.add(Dense(16, activation='sigmoid', kernel_regularizer=l1_l2(l1=0.0001, l2 = 0.0001)))
model.add(Dense(4, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))

adam = keras.optimizers.adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam)
class_weight = {0: 0.08,
               1 : 1-0.08}

model.fit(X_train_scaled, y_train, batch_size=32, epochs=500, verbose=1, shuffle=True,
          validation_data=(X_test_scaled,y_test), class_weight=class_weight)

Train on 6868 samples, validate on 2345 samples
Epoch 1/500
6868/6868 [==============================] - 1s 167us/step - loss: 0.1107 - val_loss: 0.7426
Epoch 2/500
6868/6868 [==============================] - 0s 50us/step - loss: 0.1058 - val_loss: 0.6937
Epoch 3/500
6868/6868 [==============================] - 0s 57us/step - loss: 0.1042 - val_loss: 0.7153
Epoch 4/500
6868/6868 [==============================] - 1s 73us/step - loss: 0.1039 - val_loss: 0.6961
Epoch 5/500
6868/6868 [==============================] - 0s 47us/step - loss: 0.1038 - val_loss: 0.7176
Epoch 6/500
6868/6868 [==============================] - 0s 53us/step - loss: 0.1037 - val_loss: 0.6942
Epoch 7/500
6868/6868 [==============================] - 0s 50us/step - loss: 0.1035 - val_loss: 0.7164
Epoch 8/500
6868/6868 [==============================] - 0s 48us/step - loss: 0.1033 - val_loss: 0.7264
Epoch 9/500
6868/6868 [==============================] - 0s 59us/step - loss: 0.1028 - val_loss: 0.7056
Epoch 10/500
68

6868/6868 [==============================] - 0s 61us/step - loss: 0.0607 - val_loss: 0.5390
Epoch 79/500
6868/6868 [==============================] - 0s 66us/step - loss: 0.0605 - val_loss: 0.4068
Epoch 80/500
6868/6868 [==============================] - 0s 63us/step - loss: 0.0602 - val_loss: 0.4434
Epoch 81/500
6868/6868 [==============================] - 0s 56us/step - loss: 0.0602 - val_loss: 0.4735
Epoch 82/500
6868/6868 [==============================] - 0s 48us/step - loss: 0.0598 - val_loss: 0.4386
Epoch 83/500
6868/6868 [==============================] - 0s 50us/step - loss: 0.0601 - val_loss: 0.4723
Epoch 84/500
6868/6868 [==============================] - 0s 53us/step - loss: 0.0602 - val_loss: 0.4066
Epoch 85/500
6868/6868 [==============================] - 0s 50us/step - loss: 0.0601 - val_loss: 0.3703
Epoch 86/500
6868/6868 [==============================] - 0s 48us/step - loss: 0.0598 - val_loss: 0.4508
Epoch 87/500
6868/6868 [==============================] - 0s 53us/st

6868/6868 [==============================] - 0s 66us/step - loss: 0.0574 - val_loss: 0.4060
Epoch 156/500
6868/6868 [==============================] - 0s 59us/step - loss: 0.0575 - val_loss: 0.4048
Epoch 157/500
6868/6868 [==============================] - 0s 58us/step - loss: 0.0574 - val_loss: 0.3735
Epoch 158/500
6868/6868 [==============================] - 0s 57us/step - loss: 0.0576 - val_loss: 0.4030
Epoch 159/500
6868/6868 [==============================] - 0s 56us/step - loss: 0.0575 - val_loss: 0.4141
Epoch 160/500
6868/6868 [==============================] - 0s 58us/step - loss: 0.0572 - val_loss: 0.4912
Epoch 161/500
6868/6868 [==============================] - 0s 52us/step - loss: 0.0575 - val_loss: 0.4264
Epoch 162/500
6868/6868 [==============================] - 0s 55us/step - loss: 0.0572 - val_loss: 0.4484
Epoch 163/500
6868/6868 [==============================] - 0s 58us/step - loss: 0.0570 - val_loss: 0.3665
Epoch 164/500
6868/6868 [==============================] - 0

6868/6868 [==============================] - 0s 45us/step - loss: 0.0554 - val_loss: 0.4871
Epoch 233/500
6868/6868 [==============================] - 0s 53us/step - loss: 0.0556 - val_loss: 0.4422
Epoch 234/500
6868/6868 [==============================] - 0s 57us/step - loss: 0.0550 - val_loss: 0.4272
Epoch 235/500
6868/6868 [==============================] - 0s 57us/step - loss: 0.0555 - val_loss: 0.3884
Epoch 236/500
6868/6868 [==============================] - 0s 46us/step - loss: 0.0550 - val_loss: 0.4076
Epoch 237/500
6868/6868 [==============================] - 0s 48us/step - loss: 0.0550 - val_loss: 0.4120
Epoch 238/500
6868/6868 [==============================] - 0s 58us/step - loss: 0.0551 - val_loss: 0.4376
Epoch 239/500
6868/6868 [==============================] - 0s 46us/step - loss: 0.0552 - val_loss: 0.4693
Epoch 240/500
6868/6868 [==============================] - 0s 51us/step - loss: 0.0548 - val_loss: 0.4989
Epoch 241/500
6868/6868 [==============================] - 0

6868/6868 [==============================] - 0s 66us/step - loss: 0.0542 - val_loss: 0.4981
Epoch 310/500
6868/6868 [==============================] - 0s 62us/step - loss: 0.0538 - val_loss: 0.3600
Epoch 311/500
6868/6868 [==============================] - 0s 62us/step - loss: 0.0535 - val_loss: 0.4288
Epoch 312/500
6868/6868 [==============================] - 0s 58us/step - loss: 0.0536 - val_loss: 0.4856
Epoch 313/500
6868/6868 [==============================] - 0s 64us/step - loss: 0.0537 - val_loss: 0.3771
Epoch 314/500
6868/6868 [==============================] - 0s 55us/step - loss: 0.0535 - val_loss: 0.3955
Epoch 315/500
6868/6868 [==============================] - 0s 55us/step - loss: 0.0534 - val_loss: 0.3568
Epoch 316/500
6868/6868 [==============================] - 0s 64us/step - loss: 0.0532 - val_loss: 0.5084
Epoch 317/500
6868/6868 [==============================] - 0s 70us/step - loss: 0.0532 - val_loss: 0.3514
Epoch 318/500
6868/6868 [==============================] - 1

6868/6868 [==============================] - 0s 57us/step - loss: 0.0522 - val_loss: 0.3658
Epoch 387/500
6868/6868 [==============================] - 0s 46us/step - loss: 0.0521 - val_loss: 0.3997
Epoch 388/500
6868/6868 [==============================] - 0s 53us/step - loss: 0.0518 - val_loss: 0.4224
Epoch 389/500
6868/6868 [==============================] - 0s 54us/step - loss: 0.0522 - val_loss: 0.4031
Epoch 390/500
6868/6868 [==============================] - 0s 56us/step - loss: 0.0518 - val_loss: 0.4125
Epoch 391/500
6868/6868 [==============================] - 0s 58us/step - loss: 0.0519 - val_loss: 0.3400
Epoch 392/500
6868/6868 [==============================] - 0s 54us/step - loss: 0.0520 - val_loss: 0.4233
Epoch 393/500
6868/6868 [==============================] - 0s 56us/step - loss: 0.0518 - val_loss: 0.5307
Epoch 394/500
6868/6868 [==============================] - 0s 55us/step - loss: 0.0519 - val_loss: 0.4017
Epoch 395/500
6868/6868 [==============================] - 0

6868/6868 [==============================] - 0s 58us/step - loss: 0.0510 - val_loss: 0.3753
Epoch 464/500
6868/6868 [==============================] - 0s 55us/step - loss: 0.0502 - val_loss: 0.3377
Epoch 465/500
6868/6868 [==============================] - 0s 48us/step - loss: 0.0508 - val_loss: 0.4295
Epoch 466/500
6868/6868 [==============================] - 0s 48us/step - loss: 0.0504 - val_loss: 0.4552
Epoch 467/500
6868/6868 [==============================] - 0s 56us/step - loss: 0.0504 - val_loss: 0.3664
Epoch 468/500
6868/6868 [==============================] - 0s 56us/step - loss: 0.0505 - val_loss: 0.3421
Epoch 469/500
6868/6868 [==============================] - 0s 56us/step - loss: 0.0508 - val_loss: 0.4033
Epoch 470/500
6868/6868 [==============================] - 0s 56us/step - loss: 0.0506 - val_loss: 0.4403
Epoch 471/500
6868/6868 [==============================] - 0s 56us/step - loss: 0.0508 - val_loss: 0.3683
Epoch 472/500
6868/6868 [==============================] - 0

In [179]:
# test and train AUC ROC
test_roc = roc_auc_score(y_test, model.predict_proba(X_test_scaled))
train_roc = roc_auc_score(y_train, model.predict_proba(X_train_scaled))

print("Test ROC:", test_roc)
print("Train ROC:", train_roc)

Test ROC: 0.9120510140888682
Train ROC: 0.929228426112229


In [180]:
recall_score(Y_test, (model.predict(X_test_scaled)>0.5)[:, -1])

0.890625